In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load dataset
df = pd.read_csv("3_levels_EC.tsv", sep="\t")
print("Columns:", df.columns.tolist())
print("Total data:", len(df))

sequences = df["Sequence"].astype(str).tolist()
labels = df["EC number"].astype(str).values


c:\Users\USER\Downloads\AI FINAL PROJECT TESTING\.venv311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda
Kolom: ['Entry', 'Entry Name', 'Protein names', 'Sequence', 'EC number']
Jumlah data: 236607


In [ ]:
model_name = "facebook/0esm2_t12_35M_UR5D"
# model_name = "facebook/esm2_t6_8M_UR50D"  # smaller model

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.to(device)
model.eval()


c:\Users\USER\Downloads\AI FINAL PROJECT TESTING\.venv311\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--facebook--esm2_t12_35M_UR50D. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of EsmModel were not initialized from the model checkpoint at facebook/

EsmModel(
  (embeddings): EsmEmbeddings(
    (word_embeddings): Embedding(33, 480, padding_idx=1)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): EsmEncoder(
    (layer): ModuleList(
      (0-11): 12 x EsmLayer(
        (attention): EsmAttention(
          (self): EsmSelfAttention(
            (query): Linear(in_features=480, out_features=480, bias=True)
            (key): Linear(in_features=480, out_features=480, bias=True)
            (value): Linear(in_features=480, out_features=480, bias=True)
            (rotary_embeddings): RotaryEmbedding()
          )
          (output): EsmSelfOutput(
            (dense): Linear(in_features=480, out_features=480, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (LayerNorm): LayerNorm((480,), eps=1e-05, elementwise_affine=True)
        )
        (intermediate): EsmIntermediate(
          (dense): Linear(in_features=480, out_features=1920, bias=True)
        )
        (output): EsmOutput(
      

In [9]:
@torch.no_grad()
def embed_batch(seq_list):
    """
    seq_list: list of str
    return: numpy array (batch_size, d_model)
    """
    tokens = tokenizer(
        seq_list,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=1024
    )
    tokens = {k: v.to(device) for k, v in tokens.items()}
    
    outputs = model(**tokens)              # last_hidden_state: [B, L, D]
    last_hidden = outputs.last_hidden_state
    mask = tokens["attention_mask"].unsqueeze(-1)  # [B, L, 1]
    
    masked = last_hidden * mask            # [B, L, D]
    summed = masked.sum(dim=1)             # [B, D]
    counts = mask.sum(dim=1)               # [B, 1]
    emb = (summed / counts).cpu().numpy()  # [B, D]
    return emb


In [ ]:
batch_size = 64  # can be increased to 128/256 if VRAM is sufficient
all_embeddings = []

n = len(sequences)
for start in range(0, n, batch_size):
    end = min(start + batch_size, n)
    batch_seqs = sequences[start:end]
    emb_batch = embed_batch(batch_seqs)
    all_embeddings.append(emb_batch)
    print(f"Embedded {end}/{n} sequences")

X = np.vstack(all_embeddings)
print("ESM2 feature shape:", X.shape)
np.save("esm2_features.npy", X)


Embedded 64/236607 sequences
Embedded 128/236607 sequences
Embedded 192/236607 sequences
Embedded 256/236607 sequences
Embedded 320/236607 sequences
Embedded 384/236607 sequences
Embedded 448/236607 sequences
Embedded 512/236607 sequences
Embedded 576/236607 sequences
Embedded 640/236607 sequences
Embedded 704/236607 sequences
Embedded 768/236607 sequences
Embedded 832/236607 sequences
Embedded 896/236607 sequences
Embedded 960/236607 sequences
Embedded 1024/236607 sequences
Embedded 1088/236607 sequences
Embedded 1152/236607 sequences
Embedded 1216/236607 sequences
Embedded 1280/236607 sequences
Embedded 1344/236607 sequences
Embedded 1408/236607 sequences
Embedded 1472/236607 sequences
Embedded 1536/236607 sequences
Embedded 1600/236607 sequences
Embedded 1664/236607 sequences
Embedded 1728/236607 sequences
Embedded 1792/236607 sequences
Embedded 1856/236607 sequences
Embedded 1920/236607 sequences
Embedded 1984/236607 sequences
Embedded 2048/236607 sequences
Embedded 2112/236607 seq

In [ ]:
# ============================
# 1. Load features & labels
# ============================
X = np.load("esm2_features.npy")   # (236607, 480)
print("Shape X:", X.shape)

df = pd.read_csv("3_levels_EC.tsv", sep="\t")
labels = df["EC number"].astype(str).values

le = LabelEncoder()
y = le.fit_transform(labels)

print("Total unique classes:", len(le.classes_))
print("Example classes:", list(le.classes_)[:20])

# ============================
# 2. Train-test split
#    (without stratify because there are very rare classes)
# ============================
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=None
)

print("Train size:", X_train.shape[0])
print("Test size :", X_test.shape[0])

# ============================
# 3. RandomForest with progress
# ============================
total_trees = 300
step = 50  # train 50 trees per loop -> 6 steps

rf = RandomForestClassifier(
    n_estimators=0,      # start from 0 trees
    warm_start=True,     # so that it doesn't reset on each fit()
    max_depth=None,
    n_jobs=-1,
    random_state=42
)

for n in tqdm(range(step, total_trees + 1, step), desc="Training RandomForest"):
    rf.n_estimators = n
    rf.fit(X_train, y_train)

# ============================
# 4. Evaluation
# ============================
y_pred = rf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Accuracy:", acc)

# Only use labels that actually appear in test/pred
present_labels = np.unique(np.concatenate([y_test, y_pred]))
print("Number of classes that appear in test/pred:", len(present_labels))

print("\nClassification report (only classes that appear in test):")
print(classification_report(
    y_test,
    y_pred,
    labels=present_labels,
    target_names=le.classes_[present_labels],
    zero_division=0
))

# ============================
# 5. Save model & encoder
# ============================
joblib.dump(rf, "rf_ec_esm2.joblib")
joblib.dump(le, "label_encoder_ec_esm2.joblib")
print("Model saved as 'rf_ec_esm2.joblib' and 'label_encoder_ec_esm2.joblib'")

Shape X: (236607, 480)
Jumlah kelas unik (total): 263
Contoh kelas: ['1.1.1', '1.1.2', '1.1.3', '1.1.5', '1.1.7', '1.1.9', '1.1.98', '1.1.99', '1.10.3', '1.10.5', '1.11.1', '1.11.2', '1.12.1', '1.12.2', '1.12.5', '1.12.7', '1.12.98', '1.12.99', '1.13.11', '1.13.12']
Train size: 189285
Test size : 47322


Training RandomForest: 100%|██████████| 6/6 [17:29<00:00, 174.96s/it]


Accuracy: 0.9659143738641647
Jumlah kelas yang muncul di test/pred: 232

Classification report (hanya kelas yang muncul di test):
              precision    recall  f1-score   support

       1.1.1       0.97      0.98      0.97      1401
       1.1.2       0.25      1.00      0.40         1
       1.1.3       0.78      0.95      0.86        19
       1.1.5       0.97      0.97      0.97        64
       1.1.9       0.00      0.00      0.00         2
      1.1.98       1.00      1.00      1.00         6
      1.1.99       0.97      0.85      0.91        40
      1.10.3       0.99      0.97      0.98       119
      1.10.5       1.00      1.00      1.00         1
      1.11.1       0.88      0.96      0.92       224
      1.11.2       1.00      0.50      0.67         6
      1.12.1       1.00      0.33      0.50         3
      1.12.2       0.00      0.00      0.00         1
      1.12.7       1.00      0.67      0.80         3
     1.12.98       1.00      0.86      0.92         7
     

XGBOOST


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from tqdm.auto import tqdm
import xgboost as xgb

# =========================
# 1. Load data
# =========================
X = np.load("esm2_features.npy")  # (236607, 480)
print("X shape:", X.shape)

df = pd.read_csv("3_levels_EC.tsv", sep="\t")
labels = df["EC number"].astype(str).values

le = LabelEncoder()
y = le.fit_transform(labels)
num_classes = len(le.classes_)

print("Total classes:", num_classes)

# =========================
# 2. Split train / test
# =========================
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=None
)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest  = xgb.DMatrix(X_test,  label=y_test)

# =========================
# 3. Params (CPU, hist)
# =========================
params = {
    "objective": "multi:softmax",
    "num_class": num_classes,
    "eval_metric": "mlogloss",
    "tree_method": "hist",     # CPU histogram (fast)
    "max_depth": 8,
    "eta": 0.1,
    "subsample": 0.9,
    "colsample_bytree": 0.9,
}

num_rounds = 300

# =========================
# 4. Manual loop + tqdm
# =========================
bst = None
pbar = tqdm(range(num_rounds), desc="Training XGBoost (CPU hist)")

for i in pbar:
    # each loop adds 1 tree
    bst = xgb.train(
        params=params,
        dtrain=dtrain,
        num_boost_round=1,
        xgb_model=bst,          # continue from previous model
        verbose_eval=False      # don't spam log "[0] train-mlogloss..."
    )
    # optional: every 20 iterations check train loss
    if (i + 1) % 20 == 0:
        eval_str = bst.eval(dtrain)  # example: 'train-mlogloss:0.27909'
        pbar.set_postfix_str(eval_str)

# =========================
# 5. Evaluate
# =========================
y_pred = bst.predict(dtest).astype(int)

acc = accuracy_score(y_test, y_pred)
print("\nTest Accuracy:", acc)

print("\nClassification report (only first few classes):")
print(classification_report(y_test, y_pred, target_names=le.classes_, zero_division=0))




c:\Users\USER\Downloads\AI FINAL PROJECT TESTING\.venv311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


X shape: (236607, 480)
Total classes: 263


Training XGBoost (CPU hist): 100%|██████████| 300/300 [2:11:15<00:00, 26.25s/it, [0]	eval-mlogloss:0.00144607293550710]  



Test Accuracy: 0.9628925235619796

Classification report (hanya beberapa kelas pertama):


ValueError: Number of classes, 234, does not match size of target_names, 263. Try specifying the labels parameter

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

# Labels that appear in y_test (and usually also in y_pred)
labels_used = np.unique(y_test)
print("Number of classes that appear in test:", len(labels_used))

# Original class names (EC number)
target_names = le.inverse_transform(labels_used)

print(classification_report(
    y_test,
    y_pred,
    labels=labels_used,
    target_names=target_names,
    zero_division=0
))


Jumlah kelas yang muncul di test: 232
              precision    recall  f1-score   support

       1.1.1       0.96      0.98      0.97      1401
       1.1.2       0.33      1.00      0.50         1
       1.1.3       0.71      0.89      0.79        19
       1.1.5       0.90      0.94      0.92        64
       1.1.9       0.00      0.00      0.00         2
      1.1.98       1.00      1.00      1.00         6
      1.1.99       0.94      0.82      0.88        40
      1.10.3       0.94      0.97      0.96       119
      1.10.5       1.00      1.00      1.00         1
      1.11.1       0.89      0.94      0.91       224
      1.11.2       1.00      0.33      0.50         6
      1.12.1       1.00      0.33      0.50         3
      1.12.2       0.00      0.00      0.00         1
      1.12.7       1.00      0.33      0.50         3
     1.12.98       0.83      0.71      0.77         7
     1.12.99       1.00      0.89      0.94         9
     1.13.11       0.92      0.93      0.93

In [3]:
# =========================
# 6. Save model
# =========================
bst.save_model("xgboost_ec_cpu_hist.json")
np.save("xgboost_label_classes.npy", le.classes_)
print("Model saved as xgboost_ec_cpu_hist.json")

Model saved as xgboost_ec_cpu_hist.json


# =====================================================
# INFERENCE: RandomForest & XGBoost Prediction
# =====================================================


In [ ]:
# =====================================================
# 1. Load trained models & label encoder
# =====================================================
import torch
import numpy as np
import joblib
import xgboost as xgb
from transformers import AutoTokenizer, AutoModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Load label encoder
label_encoder = joblib.load("label_encoder_ec_esm2.joblib")
num_classes = len(label_encoder.classes_)
print("Num classes:", num_classes)

# Load RandomForest model
rf_model = joblib.load("rf_ec_esm2.joblib")
print("RandomForest model loaded.")

# Load XGBoost model
xgb_model = xgb.Booster()
xgb_model.load_model("xgboost_ec_cpu_hist.json")
print("XGBoost model loaded.")


In [ ]:
# =====================================================
# 2. Load ESM2 model for embedding
# =====================================================
MODEL_NAME = "facebook/esm2_t12_35M_UR50D"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
esm_model = AutoModel.from_pretrained(MODEL_NAME)
esm_model.to(device)
esm_model.eval()

print("Loaded ESM2 model:", MODEL_NAME)

# Get input dimension from saved features
features = np.load("esm2_features.npy")
INPUT_DIM = features.shape[1]
print("INPUT_DIM:", INPUT_DIM)


In [ ]:
# =====================================================
# 3. Functions to embed sequence & predict
# =====================================================
@torch.no_grad()
def embed_seq(seq: str) -> np.ndarray:
    """
    Embed 1 protein sequence using ESM2 t12 mean pooling.
    seq: amino acid string (A,C,D,...), without spaces.
    return: numpy array shape (INPUT_DIM,)
    """
    tokens = tokenizer(
        [seq],
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=1024
    )
    tokens = {k: v.to(device) for k, v in tokens.items()}

    outputs = esm_model(**tokens)           # last_hidden_state: [1, L, D]
    last_hidden = outputs.last_hidden_state # [1, L, D]
    mask = tokens["attention_mask"].unsqueeze(-1)  # [1, L, 1]

    masked = last_hidden * mask             # [1, L, D]
    summed = masked.sum(dim=1)              # [1, D]
    counts = mask.sum(dim=1)                # [1, 1]
    emb = (summed / counts).cpu().numpy()[0]    # [D]

    # Check dimension
    if emb.shape[0] != INPUT_DIM:
        raise ValueError(
            f"Dimension embeddings ({emb.shape[0]}) != INPUT_DIM ({INPUT_DIM}). "
            f"Please retrain the models with the correct INPUT_DIM."
        )

    return emb

def predict_ec_rf(seq: str, top_k: int = 3):
    """
    Predict EC number using RandomForest.
    seq  : amino acid string (A,C,D,...)
    top_k: show top N predictions
    return: (pred_ec, conf)
    """
    emb = embed_seq(seq)  # (D,)
    emb = emb.reshape(1, -1)  # (1, D)

    # Predict probabilities
    probs = rf_model.predict_proba(emb)[0]  # (num_classes,)

    top_idx = probs.argsort()[::-1][:top_k]
    top_ec = label_encoder.inverse_transform(top_idx)
    top_conf = probs[top_idx]

    print("Top predictions (RandomForest):")
    for ec, p in zip(top_ec, top_conf):
        print(f"  {ec:10s}  (confidence = {p:.4f})")

    return top_ec[0], top_conf[0]

def predict_ec_xgb(seq: str, top_k: int = 3):
    """
    Predict EC number using XGBoost.
    seq  : amino acid string (A,C,D,...)
    top_k: show top N predictions
    return: (pred_ec, conf)
    """
    emb = embed_seq(seq)  # (D,)
    
    # XGBoost uses DMatrix
    dtest = xgb.DMatrix(emb.reshape(1, -1))
    
    # Predict probabilities
    probs = xgb_model.predict(dtest, output_margin=False)[0]  # (num_classes,)

    top_idx = probs.argsort()[::-1][:top_k]
    top_ec = label_encoder.inverse_transform(top_idx)
    top_conf = probs[top_idx]

    print("Top predictions (XGBoost):")
    for ec, p in zip(top_ec, top_conf):
        print(f"  {ec:10s}  (confidence = {p:.4f})")

    return top_ec[0], top_conf[0]


In [ ]:
# =====================================================
# 4. Test with input sequence (RandomForest)
# =====================================================
TEST_SEQ = input("Input sequence protein (A,C,D,... without space): ").strip()
print("\n=== RandomForest Prediction ===")
pred_ec_rf, conf_rf = predict_ec_rf(TEST_SEQ, top_k=3)

print("\nFinal prediction (RandomForest):")
print("Predicted EC :", pred_ec_rf)
print("Confidence   :", conf_rf)


In [ ]:
# =====================================================
# 5. Test with input sequence (XGBoost)
# =====================================================
print("\n=== XGBoost Prediction ===")
pred_ec_xgb, conf_xgb = predict_ec_xgb(TEST_SEQ, top_k=3)

print("\nFinal prediction (XGBoost):")
print("Predicted EC :", pred_ec_xgb)
print("Confidence   :", conf_xgb)
